## Advent of Code 2022

In [1]:
from aocd import get_data

### Day One

In [2]:
data = get_data(day=1, year=2022)

In [3]:
# Part One

elves = [elf.split("\n") for elf in data.split("\n\n")]
calories = [sum([int(calories) for calories in elf]) for elf in elves]
max(calories)

68787

In [4]:
# Part Two

sum(sorted(calories, reverse=True)[:3])

198041

### Day Two

In [5]:
data = get_data(day=2, year=2022)

In [6]:
# Part One

games = [(line[0], line[2]) for line in data.splitlines()]
translate = {"A": "rock", "B": "paper", "C": "scissors", "X": "rock", "Y": "paper", "Z": "scissors"}
games = [(translate[game[0]], translate[game[1]]) for game in games]

shape_scores = [{"rock": 1, "paper": 2, "scissors": 3}[game[1]] for game in games]
winners = [("rock", "paper"), ("paper", "scissors"), ("scissors", "rock")] # Loser < Winner
outcome_scores = [6 if game in winners else 3 if game[0]==game[1] else 0 for game in games]

sum([shape+outcome for (shape, outcome) in zip(shape_scores, outcome_scores)])

13924

In [7]:
# Part Two

games = [(line[0], line[2]) for line in data.splitlines()]
translate = {"A": "rock", "B": "paper", "C": "scissors", "X": "lose", "Y": "draw", "Z": "win"}
games = [(translate[game[0]], translate[game[1]]) for game in games]

shape_scores = {"rock": 1, "paper": 2, "scissors": 3}
winners = ["paper", "scissors", "rock"]
losers = ["rock", "paper", "scissors"]

def score(game):
    if game[1]=="draw":
        return (3, shape_scores[game[0]])
    elif game[1]=="win":
        return (6, shape_scores[winners[losers.index(game[0])]])
    elif game[1]=="lose":
        return (0, shape_scores[losers[winners.index(game[0])]])

scores = [score(game) for game in games]
sum([outcome + shape for (outcome, shape) in scores])

13448

In [8]:
# Part Two with a deque instead of winners/losers lists

from collections import deque

games = [(line[0], line[2]) for line in data.splitlines()]

shape_scores = {"A": 1, "B": 2, "C": 3}
winners = deque(["A", "B", "C"])

def score(game):
    while winners.index(game[0])!=1:
        winners.rotate() # Rotate until opponent's choice in middle
    if game[1]=="X":
        return shape_scores[winners[0]]
    elif game[1]=="Y":
        return 3 + shape_scores[winners[1]]
    elif game[1]=="Z":
        return 6 + shape_scores[winners[2]]

sum([score(game) for game in games])

13448

### Day Three

In [9]:
data = get_data(day=3, year=2022)

In [10]:
# Part One

import string

sacks = data.splitlines()
common_items = [set(sack[:len(sack)//2]) & set(sack[len(sack)//2:]) for sack in sacks]
alphabet = string.ascii_lowercase + string.ascii_uppercase
sum([alphabet.index(item)+1 for item, in common_items])

7889

In [11]:
# Part Two

def score(i):
    badge, = set(sacks[i-1]) & set(sacks[i]) & set(sacks[i+1])
    return alphabet.index(badge)+1

sum([score(i) for i in range(1, len(sacks)+1, 3)])

2825

In [12]:
# Part Two -- bit clearer

def score(group):
    badge, = set(group[0]) & set(group[1]) & set(group[2])
    return alphabet.index(badge)+1

sum([score(sacks[i:i+3]) for i in range(0, len(sacks), 3)])

2825

In [13]:
# Part Two with `chunked` from more_itertools

from more_itertools import chunked

def score(group):
    badge, = set(group[0]) & set(group[1]) & set(group[2])
    return alphabet.index(badge)+1

sum([score(group) for group in chunked(sacks, 3)])

2825

### Day Four

In [14]:
data = get_data(day=4, year=2022)

In [15]:
# Part One

pairs = [pair.split(",") for pair in data.splitlines()]
edges = [[edge.split("-") for edge in elf] for elf in pairs]
ranges = [[set(range(int(e[0]), int(e[1])+1)) for e in edge] for edge in edges]
sum([1 for elf in ranges if elf[0] <= elf[1] or elf[0] >= elf[1]])

431

In [16]:
# Part Two

sum([1 for elf in ranges if elf[0] & elf[1]])

823

### Day Five

In [17]:
data = get_data(day=5, year=2022)

In [18]:
# Part One

import re
import copy

""" # First approach to parsing stacks with regex -- but prefer reading nth char as below
stacks, rearrangement = data.split("\n\n")
stack_count = int((max(re.findall(r"\d+", stacks.splitlines()[-1]))))
stacks = [re.finditer(r"\[([A-Z])\]", stack) for stack in stacks.splitlines()[:-1]]
crates = {i+1: [] for i in range(stack_count)}

for stack in stacks:
    for crate in stack:
        crates[int(crate.start()/4)+1].append(crate.groups()[0])
"""

stacks, instructions = data.split("\n\n")
stacks = [list(stack[1::4]) for stack in stacks.splitlines()[:-1]] # Read every 4th char
stacks = [list(stack) for stack in zip(*stacks)] # Transpose list of lists
stacks = {k+1: [crate for crate in stack if not crate.isspace()] for k, stack in enumerate(stacks)}

stacks_rearranged = copy.deepcopy(stacks)
        
for instruction in instructions.splitlines():
    quantity, source, target = [int(i) for i in re.findall(r"\d+", instruction)]
    for q in range(quantity):
        stacks_rearranged[target].insert(0, stacks_rearranged[source].pop(0))

"".join([stacks_rearranged[k][0] for k in sorted(stacks_rearranged.keys())])

'BZLVHBWQF'

In [19]:
# Part Two

stacks_rearranged = copy.deepcopy(stacks)

for instruction in instructions.splitlines():
    quantity, source, target = [int(i) for i in re.findall(r"\d+", instruction)]
    stacks_rearranged[target][:0] = stacks_rearranged[source][:int(quantity)]
    stacks_rearranged[source] = stacks_rearranged[source][int(quantity):]

"".join([stacks_rearranged[k][0] for k in sorted(stacks_rearranged.keys())])

'TDGJQTZSL'

## Day Six

In [20]:
data = get_data(day=6, year=2022)

In [21]:
from more_itertools import windowed

marker = [marker for marker in windowed(data, 4) if len(set(marker))==4][0]
data.find("".join(marker))+4

1802

In [22]:
marker = [marker for marker in windowed(data, 14) if len(set(marker))==14][0]
data.find("".join(marker))+14

3551